# 카카오 지도/로컬 API
- 도로명 주소 --> 위도, 경도 구하기

In [1]:
from google.colab import files
up = files.upload()

Saving kakaomapkey.txt to kakaomapkey.txt


In [2]:
with open('kakaomapkey.txt') as f:
    kakao_key = f.read()

### 카카오 로컬 API 활용하기

In [3]:
import requests
from urllib.parse import quote

In [4]:
addr = '강원도 강릉시 경포로 365(저동)'
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EA%B0%95%EC%9B%90%EB%8F%84%20%EA%B0%95%EB%A6%89%EC%8B%9C%20%EA%B2%BD%ED%8F%AC%EB%A1%9C%20365%28%EC%A0%80%EB%8F%99%29'

In [5]:
# Header: "Authorization: KakaoAK {REST_API_KEY}"
result = requests.get(url,
                      headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
result

{'documents': [{'address': {'address_name': '강원 강릉시 저동 94',
    'b_code': '4215013700',
    'h_code': '4215066500',
    'main_address_no': '94',
    'mountain_yn': 'N',
    'region_1depth_name': '강원',
    'region_2depth_name': '강릉시',
    'region_3depth_h_name': '경포동',
    'region_3depth_name': '저동',
    'sub_address_no': '',
    'x': '128.896538649542',
    'y': '37.7955687639648'},
   'address_name': '강원 강릉시 경포로 365',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '강원 강릉시 경포로 365',
    'building_name': '경포대',
    'main_building_no': '365',
    'region_1depth_name': '강원',
    'region_2depth_name': '강릉시',
    'region_3depth_name': '저동',
    'road_name': '경포로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '128.896538649542',
    'y': '37.7955687639648',
    'zone_no': '25461'},
   'x': '128.896538649542',
   'y': '37.7955687639648'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [10]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
lng, lat

(128.896538649542, 37.7955687639648)

### 관광명소.csv 파일의 명소정보에 위도, 경도 추가하기

In [11]:
up = files.upload()

Saving 관광명소.csv to 관광명소.csv


In [12]:
import pandas as pd
df = pd.read_csv('관광명소.csv')
df

,명소,주소
0,남이섬,강원도 춘천시 남산면 남이섬길 1
1,설악산국립공원,강원도 인제군 북면 설악로 4193
2,강릉경포대,강원도 강릉시 경포로 365(저동)
3,오대산국립공원,강원도 강릉시 연곡면 소금강길 500
4,평창휘닉스파크,강원도 평창군 봉평면 태기로 227
5,정동진역,강원도 강릉시 강동면 정동역길 17
6,환선굴휴게소,강원도 삼척시 미로면 강원남부로 3969
7,울산바위,"강원도 속초시 울산바위길 9-17(노학동, 하이디 울산바위)"
8,주문진,강원도 강릉시 주문진읍 주문진서당길 5
9,촛대바위,강원도 동해시 촛대바위길 16-2(추암동)


In [14]:
lat_list, lng_list = [], []
for i in df.index:
    url = f"{local_url}?query={quote(df['주소'][i])}"
    result = requests.get(url,
                headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lat_list.append(lat)
    lng_list.append(lng)

In [15]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head()

,명소,주소,위도,경도
0,남이섬,강원도 춘천시 남산면 남이섬길 1,37.790125,127.526129
1,설악산국립공원,강원도 인제군 북면 설악로 4193,38.119418,128.341897
2,강릉경포대,강원도 강릉시 경포로 365(저동),37.795569,128.896539
3,오대산국립공원,강원도 강릉시 연곡면 소금강길 500,37.813571,128.704509
4,평창휘닉스파크,강원도 평창군 봉평면 태기로 227,37.590659,128.339065


### 지도위에 관광명소 표시하기

In [16]:
import folium

In [18]:
map = folium.Map(
    location=[df.위도.mean(), df.경도.mean()], zoom_start=9
)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        tooltip=df.명소[i],
        popup=folium.Popup(df.주소[i], max_width=200)
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">강원도 관광명소</h3>' 
map.get_root().html.add_child(folium.Element(title_html))

map

In [19]:
df.to_csv('관광명소2.csv', index=False)
map.save('강원도_관광명소.html')